In [3]:
import langgraph


In [4]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool
from pydantic import BaseModel, Field
from langgraph.graph import MessagesState,StateGraph, END
from langgraph.prebuilt import ToolNode
from langchain_core.messages import HumanMessage

In [117]:
from dotenv import load_dotenv
load_dotenv()

True

In [102]:
import os 
os.environ['GROQ_API_KEY']=os.getenv('GROQ_API_KEY')

In [118]:
os.environ['GOOGLE_API_KEY']=os.getenv('GOOGLE_API_KEY')


In [103]:
os.environ['UNSTRUCTURED_API_KEY']=os.getenv('UNSTRUCTURED_API_KEY')

In [104]:
os.environ['LANGCHAIN_API_KEY'] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] ="https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [105]:
class Categorization(BaseModel):
    """Respond to the user with this"""
    summarize_content: str = Field(description="give me a meaningful consise summarization of the document")
    person_name: str = Field(description="Name of the person in the document")
    govertment_id: str = Field(description="Government ID of the person in the document")
    email_address: str = Field(description="Email address of the person in the document")
    type_of_document: str = Field(description=''' the document is of what type, cho0se from the folllowing
                                  credit card,
                                  savings account,
                                  driver's license,
                                  state/country identification,
                                  passport,
                                  income statements/paystubs,
                                  tax returns,
                                  Receipts''')

In [106]:
# Inherit 'messages' key from MessagesState, which is a list of chat messages
class AgentState(MessagesState):
    # Final structured response from the agent
    final_response: Categorization

In [107]:
from langchain_groq import ChatGroq
llm=ChatGroq(model_name="Gemma2-9b-It")

In [108]:
model_with_structured_output = llm.with_structured_output(Categorization)

In [72]:
file_path="/Users/guru/appian_ai/WhatsApp Image 2024-12-21 at 22.33.39.pdf"

In [124]:
file_path1="/Users/guru/appian_ai/test1.pdf"

In [125]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(file_path1)
pages = []
async for page in loader.alazy_load():
    pages.append(page)

In [126]:
len(pages)

2

In [93]:
from langchain_unstructured import UnstructuredLoader

loader = UnstructuredLoader(
    file_path=file_path,
    strategy="hi_res",
    partition_via_api=True,
    coordinates=True,
)
docs = []
for doc in loader.lazy_load():
    docs.append(doc)

INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 200 OK"


In [115]:
first_page_docs = [doc for doc in docs if doc.metadata.get("page_number") == 1]

for doc in first_page_docs:
    print(doc.page_content)

Indian U ving Licence nt Of Tai Nadu 2. . = 0240007935 th 4, il oty o iy H 01082024 18-10-2044. 3§ . Name: PARAMAGURU H Son/Daughter Wi of: Date of Bith: 19.10.2004 HARMURUGAN £ N 76A RAJA NANDAGOPAL L=


In [100]:
content1=docs[0].page_content


In [133]:
content=''

In [134]:
for page in pages:
    content= page.page_content + content

In [135]:
print(content)

Email address : paramaguruvh@gmail.com
Type of document :
     Drivers licence
NAME:PARAMAGURU
Govermen od: 12345687



In [109]:
print(f"{pages[0].metadata}\n")
print(pages[0].page_content)

{'source': '/Users/guru/appian_ai/WhatsApp Image 2024-12-21 at 22.33.39.pdf', 'page': 0}




In [110]:
cont=pages[0].page_content

In [111]:
model_with_structured_output.invoke(content1)

INFO: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Categorization(summarize_content='Indian U ving Licence nt Of Tai Nadu 2. . = 0240007935 th 4, il oty o iy H 01082024 18-10-2044. 3§ . Name: PARAMAGURU H Son/Daughter Wi of: Date of Bith: 19.10.2004 HARMURUGAN £ N 76A RAJA NANDAGOPAL L=', person_name='PARAMAGURU H', govertment_id='0240007935', email_address='', type_of_document="driver's license")

In [119]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [137]:
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [138]:
model_struct = model.with_structured_output(Categorization)

In [139]:
content

'Email address : paramaguruvh@gmail.com\nType of document :\n     Drivers licence\nNAME:PARAMAGURU\nGovermen od: 12345687\n'

In [140]:
content1

'Indian U ving Licence nt Of Tai Nadu 2. . = 0240007935 th 4, il oty o iy H 01082024 18-10-2044. 3§ . Name: PARAMAGURU H Son/Daughter Wi of: Date of Bith: 19.10.2004 HARMURUGAN £ N 76A RAJA NANDAGOPAL L='

In [149]:
a=model_struct.invoke(content)

In [152]:
print(a.type_of_document)

driver's license


In [157]:
market_analyzer_prompt = (
    " You are a stock market news analyst. Given a list of companies,\n"
    " you search for the news articles and expert analysis related to"
    " these companies. Respond with the news summaries and expert"
    " analysis for each company."
    " Top Companies = {company_names}."
    " Use Top Companies as input parameters only if user does not provide any specific company in the query."
    " Use get_news_articles and get_expert_analysis tools to get the news articles and expert analysis."
    " After you finish, you should pass the conversation to the supervisor."
    " You should always reply with the context of responding to the user's query."
)


In [158]:
type(market_analyzer_prompt)

str

In [159]:
print(market_analyzer_prompt)

 You are a stock market news analyst. Given a list of companies,
 you search for the news articles and expert analysis related to these companies. Respond with the news summaries and expert analysis for each company. Top Companies = {company_names}. Use Top Companies as input parameters only if user does not provide any specific company in the query. Use get_news_articles and get_expert_analysis tools to get the news articles and expert analysis. After you finish, you should pass the conversation to the supervisor. You should always reply with the context of responding to the user's query.


In [ ]:
'''    
    uploaded_file = st.file_uploader("Upload the Document",type=["txt","pdf"])
    if uploaded_file is not None:
        delete_previous_file(save_dir)
        save_path = os.path.join(save_dir, uploaded_file.name)
        with open(save_path, "wb") as f:
            f.write(uploaded_file.getbuffer())
        st.success(f"File saved successfully: {uploaded_file.name}")
    submit=st.button("Submit")
    if submit:
        st.text("Summarizing and Categorising the document...")
        #content=preprocessed_data(save_dir)
        #mybot=Chatbot()
        #workflow=mybot(save_dir)
        #response = workflow.invoke({"messages":[content]})
        #st.write(response['messages'][-1].content)
        output=model()
        st.write (output)
        st.success("Document Summarized and Categorised Successfully")
        confirm = st.button("confirm")
        if confirm:
            st.text("Ingesting the data to the database...")
            st.write(output)
            st.success("Data Ingested to the database Successfully")'''